In [1]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import plotly.express    as px
import random

import plotly.io as pio
pio.renderers.default = 'vscode'

### Control Panel

In [2]:
n_submission = 1000
n_essay = 10

q1_dist = "alluniform"
q2_dist = "uniform"
q3_dist = "uniform"

# q1_dist = "gaussian"
# q2_dist = "gaussian"
# q3_dist = "gaussian"

q1_gaussian_mu = 2.5
q1_gaussian_sigma = 5.
q2_gaussian_mu = 7.
q2_gaussian_sigma = 12.
q3_gaussian_mu = 5.
q3_gaussian_sigma = 1.



### Functions

In [3]:
def generate_random_gaussian_in_range(mean=4.5, std_dev=2., lower_bound=0., upper_bound=9.):
    while True:
        num = np.random.normal(mean, std_dev)
        if lower_bound < num < upper_bound:
            return num

def plot_result(df_in, x_label="x", y_label="y", title=""):

    df = df_in.copy()
    # Add a trendline using numpy
    z = np.polyfit(df['x'], df['y'], 1)  # Linear regression
    p = np.poly1d(z)
    df['trendline'] = p(df['x'])

    fig = px.scatter(df, x='x', y='y', color='y', size='y',
                     labels={'x':x_label, 'y':y_label}, title=title
                     )
    fig.add_scatter(x=df['x'], y=df['trendline'], mode='lines', name='Trendline')
    fig.show()
    return

In [4]:

avg_e = 1.     # Maximum avg_e which means perfect English language
avg_s = 0.2    # Minimum avg_s based on the documentation (avg_s = max(avg_s, 0.2))

final_score_vec = []
avg_h_vec = []
min_v_vec = []
max_final_score = 0
for _ in range(n_submission):
    q1_vec = []
    q2_vec = []
    q3_vec = []
    aqs_vec = []    # avg_quality_scores vector
    hs_vec = []     # horizontal_stdevs vector
    for _ in range(n_essay):
        if q1_dist.lower() == "alluniform":
            q_vec = np.round([random.uniform(0, 9) for _ in range(3)], 2)
        else:
            if q1_dist.lower() == "uniform":
                q1 = np.round(random.uniform(0, 9), 2)
            elif q1_dist.lower() == "gaussian":
                q1 = np.round(generate_random_gaussian_in_range(q1_gaussian_mu, q1_gaussian_sigma), 2)
            
            if q2_dist.lower() == "uniform":
                q2 = np.round(random.uniform(0, 9), 2)
            elif q2_dist.lower() == "gaussian":
                q2 = np.round(generate_random_gaussian_in_range(q2_gaussian_mu, q2_gaussian_sigma), 2)
            
            if q3_dist.lower() == "uniform":
                q3 = np.round(random.uniform(0, 9), 2)
            elif q3_dist.lower() == "gaussian":
                q3 = np.round(generate_random_gaussian_in_range(q3_gaussian_mu, q3_gaussian_sigma), 2)
            
            q_vec = [q1, q2, q3]

        q1_vec.append(q_vec[0])
        q2_vec.append(q_vec[1])
        q3_vec.append(q_vec[2])
        aqs_vec.append(np.round(np.mean(q_vec), 2))
        hs_vec.append(np.round(np.var(q_vec), 2))

    # Calculate vertical_stdevs
    vs1 = np.round(np.var(q1_vec), 2)
    vs2 = np.round(np.var(q2_vec), 2)
    vs3 = np.round(np.var(q3_vec), 2)

    avg_q = np.round(np.mean(aqs_vec), 2)
    avg_h = np.round(np.mean(hs_vec) , 2)
    min_v = min(vs1, vs2, vs3)

    final_score = (avg_h * min_v * avg_e) / (avg_s * (9. - avg_q))
    final_score_vec.append(final_score)
    avg_h_vec.append(avg_h)
    min_v_vec.append(min_v)

    if final_score > max_final_score:
        max_final_score = final_score
        res = pd.DataFrame()
        res["q1"] = q1_vec
        res["q2"] = q2_vec
        res["q3"] = q3_vec
        res["avg_quality_scores"] = aqs_vec
        res["horizontal_stdevs"] = hs_vec

print("Max Score:", max_final_score)


Max Score: 75.83685503685503


In [5]:
tmp = pd.DataFrame()
tmp["x"] = avg_h_vec
tmp["y"] = final_score_vec
tmp.sort_values(by=['x'], inplace=True)
plot_result(tmp, "avg_h", "Final Score", "Score as a function of horizontal variance")

tmp = pd.DataFrame()
tmp["x"] = min_v_vec
tmp["y"] = final_score_vec
tmp.sort_values(by=['x'], inplace=True)
plot_result(tmp, "min_v", "Final Score", "Score as a function of vertical variance")


In [6]:
df = pd.DataFrame()
df["avg_h"] = avg_h_vec
df["min_v"] = min_v_vec
df["final_score"] = final_score_vec

print(max(final_score_vec))

fig = px.scatter_3d(df, x='avg_h', y='min_v', z='final_score',
                    color=df['final_score'], title="3D Scatter Plot")
fig.show()

75.83685503685503


### Example (The Max Final Score)

In [12]:
vs1 = np.var(res["q1"])
vs2 = np.var(res["q2"])
vs3 = np.var(res["q3"])


avg_q = np.round(np.mean(res["avg_quality_scores"]), 2)
avg_h = np.round(np.mean(res["horizontal_stdevs"]) , 2)
min_v = np.round(min(vs1, vs2, vs3), 2)

final_score = (avg_h * min_v * avg_e) / (avg_s * (9. - avg_q))

display(res)

print("Vertical stdev for q1 =", vs1)
print("Vertical stdev for q2 =", vs2)
print("Vertical stdev for q3 =", vs3)
print("="*40)
print("avg_h =", avg_h)
print("min_v =", min_v)
print("avg_q =", avg_q)
print("="*40)
print("final_score =", final_score)


,q1,q2,q3,avg_quality_scores,horizontal_stdevs
0,3.93,6.12,0.29,3.45,5.78
1,8.88,0.73,0.07,3.23,16.05
2,6.99,8.62,0.95,5.52,10.89
3,3.49,7.47,8.95,6.64,5.32
4,3.39,8.26,5.19,5.61,4.04
5,6.80,0.61,1.80,3.07,7.19
6,8.17,6.82,8.89,7.96,0.74
7,8.24,4.53,0.89,4.55,9.00
8,8.76,0.42,6.76,5.31,12.64
9,0.36,7.75,3.76,3.96,9.12


Vertical stdev for q1 = 7.636729
Vertical stdev for q2 = 10.048760999999999
Vertical stdev for q3 = 10.992925
avg_h = 8.08
min_v = 7.64
avg_q = 4.93
final_score = 75.83685503685503
